# Posture correction

En este notebook se presenta el trabajo realizado para crear los mensajes que el asistente genera en base a la información del video

In [ ]:
import os
import time
import pickle
import mediapipe as mp
import numpy as np
import pandas as pd
import ollama
workpath = 'C:/Users/Legion/TFM/Tareas/CalistenIA'
os.chdir(workpath)
import warnings
warnings.filterwarnings("ignore")

## Approach:

En un principio se cambian un poco las funciones creadas en un principio para el pose tracking, consiguiendo un set de datos con información para asistir en la postura de los ejercicios. Esta información viene dada por las distancias entre puntos de referencia simétricos (distancia entre manos, distancia entre hombros, etc.) y los ángulos formados por los distintos puntos de referencia. Con estos datos, tendremos información importante como es el rango para los ángulos, y las posiciones relativas entre puntos de referencia. Dentro de las posibilidades de este proyecto, estos datos permiten corregir posturas de manera heurística en base a referencias para cada ejercicios en cuanto a cómo ejecutar el ejercicio con postura correcta

In [ ]:
def kpis_excercise(excercise_kpi):
    df_useful = excercise_kpi.rename(columns={
        'WRISTS_DISTANCE': 'DISTANCE BETWEEN HANDS', 'ELBOWS_DISTANCE': 'DISTANCE BETWEEN ELBOWS',
        'SHOULDERS_DISTANCE': 'DISTANCE BETWEEN SHOULDERS', 'HIPS_DSITANCE': 'DISTANCE BETWEEN HIPS',
        'KNEES_DISTANCE': 'DISTANCE BETWEEN KNEES', 'ANKLES_DISTANCE': 'DISTANCE BETWEEN FEET',
        'LEFT_ELBOW_ANGLE': 'LEFT ELBOW ANGLE','RIGHT_ELBOW_ANGLE': 'RIGHT ELBOW ANGLE',
        'LEFT_SHOULDER_ANGLE': 'LEFT SHOULDER ANGLE','RIGHT_SHOULDER_ANGLE': 'RIGHT SHOULDER ANGLE',
        'LEFT_HIP_ANGLE': 'LEFT HIP ANGLE','RIGHT_HIP_ANGLE': 'RIGHT HIP ANGLE',
        'LEFT_KNEE_ANGLE': 'LEFT KNEE ANGLE','RIGHT_KNEE_ANGLE': 'RIGHT KNEE ANGLE'})
    description_values = df_useful.describe().loc[['min', 'max', 'mean', '25%', '50%', '75%']]
    return description_values

# USING KPIS TO EFFECTIVELY PROMPT A POSTURE CORRECTION

# Asegurar que un angulo se mantiene dentro de un intervalo de referencia
def angle_in_range(angle, ang_min, ang_max):
    if ang_min <= angle <= ang_max:
        is_correct = True
    else:
        is_correct = False
    return is_correct

# Evaluar que dos distancias son iguales con un rango de error del 15% de la distancia
def equal_distances(dist_1, dist_2):
    min_dist = dist_1 * 0.85
    max_dist = dist_1 * 1.15
    
    if min_dist <= dist_2 <= max_dist:
        is_equal_answer = 'The distances are equal'
    else:
        is_equal_answer = 'The distances differ more than they should'
    return is_equal_answer

# Evaluar que un ángulo de referencia se encuentra en el rango efectivo para un ejercicio 
# teniendo en cuenta posible error que puede haber por el procesado del video ne base a la perspectiva

def effective_angle_range(min_vid, max_vid, min_small_angle, max_small_angle, min_big_angle, max_big_angle):
    is_min_angle_vid_correct = angle_in_range(min_vid, min_small_angle, max_small_angle)
    is_max_angle_vid_correct = angle_in_range(max_vid, min_big_angle, max_big_angle)
    if is_min_angle_vid_correct and is_max_angle_vid_correct:
        angle_response = 'Angle movement within the correct range.'
    elif is_max_angle_vid_correct:
        angle_response = 'Angle movement outside the range (lower than minimum angle).'
    elif is_min_angle_vid_correct:
        angle_response = 'Angle movement outside the range (higher than maximum angle).'
    else: 
        angle_response = f'Angle movement outside the range (range seen: [{min_vid} - {max_vid}]).'
    return angle_response

# Evaluar datos y sacar respuesta para Sentadillas
def squat_prompt(squat_kpis):
    left_knee = effective_angle_range(squat_kpis['LEFT KNEE ANGLE']['min'],
                                      squat_kpis['LEFT KNEE ANGLE']['max'],
                                      80, 110,
                                      170, 190)
    right_knee = effective_angle_range(squat_kpis['RIGHT KNEE ANGLE']['min'],
                                      squat_kpis['RIGHT KNEE ANGLE']['max'],
                                      80, 110,
                                      170, 190)
    left_hip = effective_angle_range(squat_kpis['LEFT HIP ANGLE']['min'],
                                      squat_kpis['LEFT HIP ANGLE']['max'],
                                      20, 90,
                                      170, 190)
    right_hip = effective_angle_range(squat_kpis['RIGHT HIP ANGLE']['min'],
                                      squat_kpis['RIGHT HIP ANGLE']['max'],
                                      20, 90,
                                      170, 190)
    shoulder_leg_distances = equal_distances(squat_kpis['DISTANCE BETWEEN FEET']['mean'],
                                             squat_kpis['DISTANCE BETWEEN SHOULDERS']['mean'])

    
    # Crear una lista de variables y sus nombres correspondientes
    kpis = [
        ('left knee angle', left_knee),
        ('right knee angle', right_knee),
        ('left hip angle', left_hip),
        ('right knee angle', right_hip),
        ('distance between shoulders compared to distance between feet', shoulder_leg_distances)
        ]

    # Prompt base
    prompt = """
                The reference for a good squat is: \n
                    - knee angles in a range between 80 degrees and 110 degrees. \n
                    - Hip angles in a range between 20 degrees and 180 degrees. \n
                    - Distance between shoulders similar to distance between feet. \n
                
                I sent a video doing the excercise.
                
                The following key points describe key points taken from the video.
                They will help evaluate the excercise.\n"""
    
    # Agregar la información de cada variable al prompt
    for kpi, state in kpis:
        prompt += f"- {kpi}: {state}\n"

    prompt += """Based on the key points presented, do you think I am doing correct squats?"""
    print('----------CREATING OLLAMA ANSWER FOR: SQUAT------------')
    response = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content': """You are a calisthenics assistant. You need to give advice to a person doing squats ('sentadillas' in spanish).
                          Based on the key points presented, give a polite answer highlighting
                          the possible bad posturing they might be doing, explaining why
                          you think it is not correct. You must end your coaching text by
                          explaining that you could have processed the video incorrectly,
                          encouraging them to send another video with a better perspective
                          to capture perfectly the movement. Give the answer in spanish."""
        },
       {
         'role': 'user',
         'content': prompt
         ,
       },
        {'role': 'assistant',
        'content': 'My answer as a calisthenics assistant:'}
     ])
    answer = response['message']['content']
    print('----------ANSWER CREATED------------')
    return answer

# Evaluar datos y sacar respuesta para Dominadas
def pullup_prompt(pullup_kpis):
    left_elbow = effective_angle_range(pullup_kpis['LEFT ELBOW ANGLE']['min'],
                                       pullup_kpis['LEFT ELBOW ANGLE']['max'],
                                       15, 45,
                                       170, 190)
    right_elbow = effective_angle_range(pullup_kpis['RIGHT ELBOW ANGLE']['min'],
                                        pullup_kpis['RIGHT ELBOW ANGLE']['max'],
                                        15, 45,
                                        170, 190)
    left_shoulder = effective_angle_range(pullup_kpis['LEFT SHOULDER ANGLE']['min'],
                                          pullup_kpis['LEFT SHOULDER ANGLE']['max'],
                                          10, 40,
                                          170, 190)
    right_shoulder = effective_angle_range(pullup_kpis['RIGHT SHOULDER ANGLE']['min'],
                                           pullup_kpis['RIGHT SHOULDER ANGLE']['max'],
                                           10, 40,
                                           170, 190)
    left_hip = effective_angle_range(pullup_kpis['LEFT HIP ANGLE']['min'],
                                     pullup_kpis['LEFT HIP ANGLE']['max'],
                                     140, 180,
                                     160, 190)
    right_hip = effective_angle_range(pullup_kpis['RIGHT HIP ANGLE']['min'],
                                      pullup_kpis['RIGHT HIP ANGLE']['max'],
                                      140, 180,
                                      160, 190)
    shoulder_hand_distances = equal_distances(pullup_kpis['DISTANCE BETWEEN HANDS']['mean'],
                                             pullup_kpis['DISTANCE BETWEEN SHOULDERS']['mean'])

    
    # Crear una lista de variables y sus nombres correspondientes
    kpis = [
        ('left elbow angle', left_elbow),
        ('right elbow angle', right_elbow),
        ('left shoulder angle', left_shoulder),
        ('right shoulder angle', right_shoulder),
        ('left hip angle', left_hip),
        ('right knee angle', right_hip),
        ('distance between hands compared to distance between shoulders', shoulder_hand_distances)
        ]

    # Prompt base
    prompt = """
                The reference for a good pullup is: \n
                    - elbow angles in a range between 20 degrees and 180 degrees for full range of motion. \n
                    - shoulder angles in a range between 20 degrees and 180 degrees. \n
                    - Hip angles in a range between 140 degrees and 190 degrees to avoid kipping. \n
                    - Distance between shoulders similar to distance between hands to avoid injuries.\n
                
                I sent a video doing the excercise.
                
                The following key points describe key points taken from the video.
                They will help evaluate the excercise.\n"""
    
    # Agregar la información de cada variable al prompt
    for kpi, state in kpis:
        prompt += f"- {kpi}: {state}\n"

    prompt += """Based on the key points presented, do you think I am doing correct pull ups?"""
    print('----------CREATING OLLAMA ANSWER FOR: PULL UP------------')
    response = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content': """You are a calisthenics assistant. You need to give advice to a person doing pull ups ('dominadas' in spanish).
                          Based on the key points presented, give a polite answer highlighting
                          the possible bad posturing they might be doing, explaining why
                          you think it is not correct. You must end your coaching motivational text by
                          explaining that you could have processed the video incorrectly,
                          encouraging them to send another video with a better perspective
                          to capture perfectly the movement. Give the answer in spanish. 
                          Answer as if you had seen the video."""
        },
       {
         'role': 'user',
         'content': prompt
         ,
       },
        {'role': 'assistant',
        'content': 'My answer as a calisthenics assistant who saw the video:'}
     ])
    answer = response['message']['content']
    print('----------ANSWER CREATED------------')
    return answer


# Evaluar datos y sacar respuesta para Fondos
def dip_prompt(dip_kpis):
    left_elbow = effective_angle_range(dip_kpis['LEFT ELBOW ANGLE']['min'],
                                       dip_kpis['LEFT ELBOW ANGLE']['max'],
                                       75, 100,
                                       170, 190)
    right_elbow = effective_angle_range(dip_kpis['RIGHT ELBOW ANGLE']['min'],
                                        dip_kpis['RIGHT ELBOW ANGLE']['max'],
                                        75, 100,
                                        170, 190)
    left_shoulder = effective_angle_range(dip_kpis['LEFT SHOULDER ANGLE']['min'],
                                          dip_kpis['LEFT SHOULDER ANGLE']['max'],
                                          0,42,
                                          45,100)
    right_shoulder = effective_angle_range(dip_kpis['RIGHT SHOULDER ANGLE']['min'],
                                           dip_kpis['RIGHT SHOULDER ANGLE']['max'],
                                           0,42,
                                           45,100)
    left_hip = effective_angle_range(dip_kpis['LEFT HIP ANGLE']['min'],
                                     dip_kpis['LEFT HIP ANGLE']['max'],
                                     140, 180,
                                     160, 190)
    right_hip = effective_angle_range(dip_kpis['RIGHT HIP ANGLE']['min'],
                                      dip_kpis['RIGHT HIP ANGLE']['max'],
                                      140, 180,
                                      160, 190)
    
    # Crear una lista de variables y sus nombres correspondientes
    kpis = [
        ('left elbow angle', left_elbow),
        ('right elbow angle', right_elbow),
        ('left shoulder angle', left_shoulder),
        ('right shoulder angle', right_shoulder),
        ('left hip angle', left_hip),
        ('right knee angle', right_hip)
        ]

    # Prompt base
    prompt = """
                The reference for a good dip is: \n
                    - elbow angles in a range between 80 degrees and 180 degrees for full range of motion. \n
                    - shoulder angles in a range between 0 degrees and 90 degrees. \n
                    - Hip angles in a range between 140 degrees and 190 degrees to avoid kipping. \n
                
                I sent a video doing the excercise.
                
                The following key points describe key points taken from the video.
                They will help evaluate the excercise.\n"""
    
    # Agregar la información de cada variable al prompt
    for kpi, state in kpis:
        prompt += f"- {kpi}: {state}\n"

    prompt += """Based on the key points presented, do you think I am doing correct dips?"""
    print('----------CREATING OLLAMA ANSWER FOR: DIPS------------')
    response = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content': """You are a calisthenics assistant. You need to give advice to a person doing dips ('fondos' in spanish).
                          Based on the key points presented, give a polite answer highlighting
                          the possible bad posturing they might be doing, explaining why
                          you think it is not correct. You must end your coaching motivational text by
                          explaining that you could have processed the video incorrectly,
                          encouraging them to send another video with a better perspective
                          to capture perfectly the movement. Give the answer in spanish. 
                          Answer as if you had seen the video."""
        },
       {
         'role': 'user',
         'content': prompt
         ,
       },
        {'role': 'assistant',
        'content': 'My answer as a calisthenics assistant who saw the video:'}
     ])
    answer = response['message']['content']
    print('----------ANSWER CREATED------------')
    return answer

# Evaluar datos y sacar respuesta para Flexiones
def pushup_prompt(pushup_kpis):
    left_elbow = effective_angle_range(pushup_kpis['LEFT ELBOW ANGLE']['min'],
                                       pushup_kpis['LEFT ELBOW ANGLE']['max'],
                                       50, 90,
                                       170, 190)
    right_elbow = effective_angle_range(pushup_kpis['RIGHT ELBOW ANGLE']['min'],
                                        pushup_kpis['RIGHT ELBOW ANGLE']['max'],
                                        50, 90,
                                        170, 190)
    left_shoulder = effective_angle_range(pushup_kpis['LEFT SHOULDER ANGLE']['min'],
                                          pushup_kpis['LEFT SHOULDER ANGLE']['max'],
                                          0,42,
                                          45,110)
    right_shoulder = effective_angle_range(pushup_kpis['RIGHT SHOULDER ANGLE']['min'],
                                           pushup_kpis['RIGHT SHOULDER ANGLE']['max'],
                                           0,42,
                                           45,110)
    left_hip = effective_angle_range(pushup_kpis['LEFT HIP ANGLE']['min'],
                                     pushup_kpis['LEFT HIP ANGLE']['max'],
                                     160, 180,
                                     160, 190)
    right_hip = effective_angle_range(pushup_kpis['RIGHT HIP ANGLE']['min'],
                                      pushup_kpis['RIGHT HIP ANGLE']['max'],
                                      160, 180,
                                      160, 190)
    shoulder_hand_distances = equal_distances(pushup_kpis['DISTANCE BETWEEN HANDS']['mean'],
                                             pushup_kpis['DISTANCE BETWEEN SHOULDERS']['mean'])
    
    # Crear una lista de variables y sus nombres correspondientes
    kpis = [
        ('left elbow angle', left_elbow),
        ('right elbow angle', right_elbow),
        ('left shoulder angle', left_shoulder),
        ('right shoulder angle', right_shoulder),
        ('left hip angle', left_hip),
        ('right knee angle', right_hip)
        ('distance between hands compared to distance between shoulders', shoulder_hand_distances)
        ]

    # Prompt base
    prompt = """
                The reference for a good dip is: \n
                    - elbow angles in a range between 60 degrees and 180 degrees for full range of motion. \n
                    - shoulder angles in a range between 0 degrees and 90 degrees. \n
                    - Hip angles in a range between 160 degrees and 190 degrees to strengthen the core. \n
                
                I sent a video doing the excercise.
                
                The following key points describe key points taken from the video.
                They will help evaluate the excercise.\n"""
    
    # Agregar la información de cada variable al prompt
    for kpi, state in kpis:
        prompt += f"- {kpi}: {state}\n"

    prompt += """Based on the key points presented, do you think I am doing correct pushups?"""
    print('----------CREATING OLLAMA ANSWER FOR: PUSH UPS------------')
    response = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content': """You are a calisthenics assistant. You need to give advice to a person doing push ups ('flexiones' in spanish).
                          Based on the key points presented, give a polite answer highlighting
                          the possible bad posturing they might be doing, explaining why
                          you think it is not correct. You must end your coaching motivational text by
                          explaining that you could have processed the video incorrectly,
                          encouraging them to send another video with a better perspective
                          to capture perfectly the movement. Give the answer in spanish. 
                          Answer as if you had seen the video."""
        },
       {
         'role': 'user',
         'content': prompt
         ,
       },
        {'role': 'assistant',
        'content': 'My answer as a calisthenics assistant who saw the video:'}
     ])
    answer = response['message']['content']
    print('----------ANSWER CREATED------------')
    return answer

# Respuesta en caso de haber problemas al procesar el video o clasificar el ejercicio.¡
def unknown_excercise():
    print('----------CREATING OLLAMA ANSWER FOR: UNKNOWN EXCERCISE------------')
    response = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'system',
            'content': """You are a calisthenics assistant. You need to give advice to a person doing an excercise.
                          They sent you a video that you could not process. Explain that you are sorry and it is an inconvinience.
                          Ask them politely to send another video, with another perspective or better light, 
                          so that you can process it correctly.
                          Give the answer in spanish. 
                          Answer as if you had seen the video."""
        },
       {
         'role': 'user',
         'content': ''' Este es mi ejercicio. Me ayudas?'''
         ,
       },
        {'role': 'assistant',
        'content': 'My answer as a calisthenics assistant that cannot understand the video:'}
     ])
    answer = response['message']['content']
    print('----------ANSWER CREATED------------')
    return answer